In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'

In [2]:
lh = pd.read_pickle(os.path.join(dir_, 'normalized_to_rating_filter_track_5_user_100.pkl'))
train = np.loadtxt(os.path.join(dir_, 'rating_matrix_normalized_to_rating_filter_track_5_user_100.csv'), delimiter=',')
pd_train = pd.read_pickle(os.path.join(dir_, 'train_normalized_to_rating_filter_track_5_user_100.pkl'))

In [3]:
num_user = len(train)
num_track = len(train[0])
print(num_user, num_track)

953 157567


In [4]:
def pearson_correlation(u1_index, u2_index):
        result = 0.0
        user1_data = train[u1_index]
        user2_data = train[u2_index]

        rx_avg = user_average_rating(user1_data)
        ry_avg = user_average_rating(user2_data)
        sxy = common_items(user1_data, user2_data)

        top_result = 0.0
        bottom_left_result = 0.0
        bottom_right_result = 0.0
        for item in sxy:
            rxs = user1_data[item]
            rys = user2_data[item]

            top_result += (rxs - rx_avg)*(rys - ry_avg)
            bottom_left_result += pow((rxs - rx_avg), 2)
            bottom_right_result += pow((rys - ry_avg), 2)
        bottom_left_result = math.sqrt(bottom_left_result)
        bottom_right_result = math.sqrt(bottom_right_result)
        
        ################################################################
        if (bottom_left_result * bottom_right_result) == 0:
            return -2, -2 # dump the data
        ################################################################
        
        result = top_result/(bottom_left_result * bottom_right_result)
        return len(sxy), result

def user_average_rating(u):
    avg_rating = 0.0
    for i in u:
        avg_rating += i
    avg_rating /= len(u) * 1.0
    return avg_rating

def common_items(u1, u2):
    result = []
    for i in range(num_track):
        if u1[i] > 0 and u2[i] > 0:
            result.append(i)
    return result

In [5]:
sim_table = pd.DataFrame(columns=['u1', 'u2', 'common', 'pearson']) 
for i in tqdm(range(num_user)):
    for j in range(i+1, num_user):
        c, p = pearson_correlation(i,j)
        if p != -2:
            sim_table = sim_table.append({'u1': i, 'u2': j, 'common': c, 'pearson': p}, ignore_index=True)
            sim_table = sim_table.append({'u1': j, 'u2': i, 'common': c, 'pearson': p}, ignore_index=True)

In [ ]:
sim_table = pd.DataFrame(columns=['u1', 'u2', 'common', 'pearson']) 
for i in tqdm(range(num_user)):
    for j in range(i+1, num_user):
        c, p = pearson_correlation(i,j)
        if p != -2:
            p = p * min(c,200) / 200
            sim_table = sim_table.append({'u1': i, 'u2': j, 'common': c, 'pearson': p}, ignore_index=True)
            sim_table = sim_table.append({'u1': j, 'u2': i, 'common': c, 'pearson': p}, ignore_index=True)

In [37]:
sim_table = pd.read_pickle(os.path.join(dir_, 'similarity.pkl'))

In [6]:
# sim_table_ = sim_table[sim_table['common']>10]
sim_table_ = sim_table.copy()
sim_table_ = sim_table_.sort_values(by=['pearson'],  ascending=False)
sim_table_ = sim_table_.reset_index()
high_similarity = sim_table_[:int(len(sim_table_)*2/10)]
low_similarity =sim_table_[int(len(sim_table_)*8/10):]
high_similarity_v = sim_table_.iloc[int(len(sim_table_)*2/10)]['pearson']
low_similarity_v = sim_table_.iloc[int(len(sim_table_)*8/10)]['pearson']
print(high_similarity_v, low_similarity_v)

0.7043162083748425 0.4932814853062143


In [7]:
pairs = pd.DataFrame(columns=['uid', 'pair']) 
for i in range(num_user):
    pair = len(high_similarity[high_similarity['u1'] == i])
    pairs = pairs.append({'uid': i, 'pair': pair}, ignore_index=True)
pairs = pairs.sort_values(by=['pair'],  ascending=True)

groups = []
group_size = 9
pairs = pairs[pairs['pair'] >= (group_size-1)]
p_status = len(pairs)
pbar = tqdm(total=len(pairs)) 
while len(pairs) >= group_size:
    group = []
    uid = pairs.iloc[0]['uid']
    pbar.update(p_status - len(pairs))
    p_status -= (p_status - len(pairs))
    pair = pairs.iloc[0]['pair']
    group.append(uid)
    if pair == (group_size-1):
        uid = high_similarity[high_similarity['u1'] == uid]
        for _, row in uid.iterrows():
            group.append(int(row['u2']))
        check_sim = True
        for i in range(1, group_size):
            for j in range(i+1, group_size):
                check = high_similarity[high_similarity['u1'] == group[i]]
                check = check[check['u2'] == group[j]]
                if len(check) == 0:
                    check_sim = False
        if check_sim:
#             print(group)
            groups.append(group)
            for u in group:
                high_similarity = high_similarity[high_similarity['u1'] != u]
                high_similarity = high_similarity[high_similarity['u2'] != u]
        else:
            high_similarity = high_similarity[high_similarity['u1'] != group[0]]
            high_similarity = high_similarity[high_similarity['u2'] != group[0]]
        pairs = pd.DataFrame(columns=['uid', 'pair']) 
        for i in range(num_user):
            pair = len(high_similarity[high_similarity['u1'] == i])
            pairs = pairs.append({'uid': i, 'pair': pair}, ignore_index=True)
        pairs = pairs.sort_values(by=['pair'],  ascending=True)
        pairs = pairs[pairs['pair'] >= (group_size-1)]
    else:
        uid = high_similarity[high_similarity['u1'] == uid]
        avaliable_uid = []
        for _, row in pairs.iterrows():
            if len(uid[uid['u2'] == row['uid']]) > 0:
                avaliable_uid.append(int(row['uid']))
                
        pointer = []
        for i in range(group_size-1):
            pointer.append(i)

        while pointer[0] < (len(avaliable_uid) - (group_size - 1) + 2) and len(avaliable_uid) >= (group_size-1):
            group_ = group.copy()
            for i in range(group_size-1):
                group_.append(avaliable_uid[pointer[i]])
            check_sim = True
            for i in range(1, group_size):
                for j in range(i+1, group_size):
                    check = high_similarity[high_similarity['u1'] == group_[i]]
                    check = check[check['u2'] == group_[j]]
                    if len(check) == 0:
                        check_sim = False
            if check_sim:
#                 print('_', group_)
                groups.append(group_)
                for u in group_:
                    high_similarity = high_similarity[high_similarity['u1'] != u]
                    high_similarity = high_similarity[high_similarity['u2'] != u]
                break
            else:
                p = 0
                added = False
                while not added:
                    if (group_size-2) - p < 0:
                        pointer[0] = (len(avaliable_uid) - (group_size - 1) + 2)
                        break
                    if pointer[(group_size-2) - p] < (len(avaliable_uid) - 1 - p):
                        pointer[(group_size-2) - p] += 1
                        while p > 0:
                            p -= 1
                            if (group_size-2) - p != 0:
                                pointer[(group_size-2) - p] = pointer[(group_size-2) - p - 1] + 1
                        added = True
                    else:
                        p += 1
        if pointer[0] == (len(avaliable_uid) - (group_size - 1) + 2) or len(avaliable_uid) < (group_size-1):
            high_similarity = high_similarity[high_similarity['u1'] != group[0]]
            high_similarity = high_similarity[high_similarity['u2'] != group[0]]
        pairs = pd.DataFrame(columns=['uid', 'pair']) 
        for i in range(num_user):
            pair = len(high_similarity[high_similarity['u1']==i])
            pairs = pairs.append({'uid': i, 'pair': pair}, ignore_index=True)
        pairs = pairs.sort_values(by=['pair'],  ascending=True)
        pairs = pairs[pairs['pair'] >= (group_size-1)]
groups

[[458, 408, 298, 548, 518, 864, 328, 739, 804],
 [486, 952, 87, 324, 651, 485, 445, 511, 799],
 [729, 643, 950, 431, 764, 241, 838, 72, 366],
 [212, 271, 128, 358, 753, 280, 912, 755, 657],
 [576, 854, 457, 131, 678, 819, 340, 658, 362],
 [738, 194, 924, 768, 156, 655, 540, 475, 599],
 [164, 505, 805, 669, 448, 235, 30, 721, 709],
 [857, 287, 834, 778, 846, 274, 477, 580, 117],
 [525, 574, 810, 105, 494, 695, 38, 493, 314],
 [470, 390, 404, 726, 419, 1, 668, 517, 95],
 [377, 285, 675, 856, 915, 158, 66, 736, 261],
 [691, 896, 744, 258, 491, 713, 375, 916, 145],
 [943, 183, 277, 766, 411, 841, 881, 939, 795],
 [190, 331, 407, 575, 911, 619, 348, 363, 773],
 [523, 227, 151, 176, 456, 200, 40, 620, 850],
 [45, 51, 327, 748, 616, 25, 801, 292, 337],
 [907, 671, 826, 24, 791, 10, 388, 332, 594],
 [659, 254, 23, 218, 353, 378, 803, 264, 342],
 [629, 33, 272, 735, 28, 290, 11, 947, 155],
 [127, 116, 217, 565, 310, 433, 527, 185, 762],
 [471, 815, 59, 20, 210, 863, 587, 91, 488],
 [69, 265, 79

In [8]:
for group in groups:
    if len(group) != group_size:
        print('Group size error')
    for i in range(group_size):
        for j in range(i+1, group_size):
            u1 = group[i]
            u2 = group[j]
            c, p = pearson_correlation(u1, u2)
            if p < high_similarity_v:
                print('Similarity error', u1, u2, p)
user_status = np.zeros(num_user)
for group in groups:
    for u in group:
        user_status[u] += 1
for i in user_status:
    if i > 1:
        print('Depulicate error')

In [9]:
len(groups)

45

In [10]:
np.savetxt(os.path.join(dir_, 'groups/high/' + str(group_size)  + '.csv'), groups, delimiter=",")

In [11]:
group_dir_ = '../data/groups/high'
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [12]:
similarities = []
for groups in tqdm(groups_n):
    n = 0
    similarity = 0
    for group in groups:
        for i in range(len(groups[0])):
            for j in range(i+1, len(groups[0])):
                u1 = int(group[i])
                u2 = int(group[j])
                c, p = pearson_correlation(u1, u2)
                p = p * min(c,200) / 200
                similarity += p
                n += 1 
    similarity /= n
    similarities.append(similarity)

In [13]:
similarities

[0.6188003099779206,
 0.6248538680883,
 0.639427261895482,
 0.6476093872573909,
 0.6550850150323636,
 0.6578850594309699,
 0.6632940125243416,
 0.6653844479861635,
 0.6655144506736822]

In [14]:
sum(similarities)/len(similarities)

0.6486504236518461